A notebook to test Pytorch.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

import numpy as np
import os
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib
import matplotlib.pyplot as plt

# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 24
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [3]:
torch.zeros(3, device=torch.device('cpu'))

tensor([0., 0., 0.])

In [4]:
w = torch.tensor([3,5,4.0])
w.dot(torch.ones(3))

tensor(12.)

## Torch 2d to 1d and vice versa

In [5]:
torch.eye(3).matmul(torch.tensor([[2., 3, 1]]).T)

tensor([[2.],
        [3.],
        [1.]])

In [6]:
torch.tensor([[2., 3, 1]]).T

tensor([[2.],
        [3.],
        [1.]])

In [7]:
V = torch.randn(3, 1)
V

tensor([[-0.8642],
        [ 0.2524],
        [ 0.2054]])

In [8]:
V.reshape(-1)

tensor([-0.8642,  0.2524,  0.2054])

In [9]:
v = torch.rand(4)
v

tensor([0.3889, 0.2695, 0.8233, 0.2854])

In [10]:
v - 1

tensor([-0.6111, -0.7305, -0.1767, -0.7146])

In [11]:
v.reshape(4, 1)

tensor([[0.3889],
        [0.2695],
        [0.8233],
        [0.2854]])

## Torch distributions
https://pytorch.org/docs/stable/distributions.html

In [12]:
import torch.distributions as dists

In [13]:
dis = dists.Normal(0, 1)
dis.log_prob(torch.ones(5))

tensor([-1.4189, -1.4189, -1.4189, -1.4189, -1.4189])

### Grad of log (Normal density)

$$ \nabla_x \log p(x) = -x$$ where $p(x) = \mathcal{N}(0,1)$.

Input is a scalar

In [14]:
x = torch.tensor([3.0], requires_grad=True)
y = 2*x
lp = dis.log_prob(y)

In [15]:
lp.backward(retain_graph=True) 
x.grad

tensor([-12.])

In [16]:
torch.autograd.grad([lp], [x, y], retain_graph=True, only_inputs=True)

(tensor([-12.]), tensor([-6.]))

Input is n x 1

In [17]:
X = torch.tensor([[-3, 1.0, 2]], requires_grad=True).T
Lp = dis.log_prob(X)
sum_lp = torch.sum(Lp)

In [18]:
torch.autograd.grad(sum_lp, X, retain_graph=True, only_inputs=True)

(tensor([[ 3.],
         [-1.],
         [-2.]]),)

## Multivariate normal

Assume $p(x) = \mathcal{N}(x \mid \mu, I)$. Then

$$ \nabla_x \log p(x) = -(x-\mu) $$

In [19]:
mdist = dists.MultivariateNormal(
    loc=torch.tensor([1, 2]), covariance_matrix=torch.eye(2))

X = torch.tensor([
    [0, 0.0],
    [0, 1.0],
    [1.0, 4],
])
X.requires_grad = True
logp = mdist.log_prob(X)
torch.autograd.grad(torch.sum(logp), X, retain_graph=True, only_inputs=True)[0]

tensor([[ 1.,  2.],
        [ 1.,  1.],
        [ 0., -2.]])

## Broadcasting

In [20]:
a = torch.arange(1, 4).reshape(3, 1)
a

tensor([[1],
        [2],
        [3]])

In [21]:
b = torch.arange(3, 7).reshape(1, 4)
b

tensor([[3, 4, 5, 6]])

In [22]:
a+b

tensor([[4, 5, 6, 7],
        [5, 6, 7, 8],
        [6, 7, 8, 9]])

In [23]:
a + b.reshape(-1)

tensor([[4, 5, 6, 7],
        [5, 6, 7, 8],
        [6, 7, 8, 9]])

In [24]:
torch.exp

<function _VariableFunctions.exp>

In [25]:
torch.zeros((3,4))

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

## Remove diagonal

In [26]:
A = torch.arange(16).reshape(4,4)
A

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])

In [27]:
diagA = torch.diagflat(torch.diag(A))
diagA

tensor([[ 0,  0,  0,  0],
        [ 0,  5,  0,  0],
        [ 0,  0, 10,  0],
        [ 0,  0,  0, 15]])

In [28]:
A - diagA

tensor([[ 0,  1,  2,  3],
        [ 4,  0,  6,  7],
        [ 8,  9,  0, 11],
        [12, 13, 14,  0]])

## Multinomial

In [29]:
d = 5
m = 20
mult = dists.multinomial.Multinomial(total_count=d, probs=torch.ones(d)/d)
mult.sample()

tensor([2., 1., 2., 0., 0.])

## Quadratic form
   

In [30]:
w = torch.rand(5)
A = torch.randn(5,5)

In [31]:
A.matmul(w).matmul(w)

tensor(1.2747)

## Bool to float

In [32]:
v = torch.randn(5)
v

tensor([-1.0508,  0.4999,  0.7936,  1.4722, -0.0437])

In [33]:
I = v >  0
I

tensor([False,  True,  True,  True, False])

In [34]:
I.to(torch.float)

tensor([0., 1., 1., 1., 0.])

## Torch scalar to normal float

In [35]:
torch_num = torch.tensor(3.78)
torch_num.item()

3.7799999713897705

## Elementwise product

In [36]:
A = torch.randn(5,2)
B = torch.rand(5, 2)
torch.sum(A*B, 1)

tensor([-0.3576, -0.2991, -0.9420, -0.3542,  1.6145])

## Meshgrid


In [37]:
a = torch.arange(5.0)
print(a)
b = torch.rand(4)
print(b)

tensor([0., 1., 2., 3., 4.])
tensor([0.1454, 0.0108, 0.2072, 0.0541])


In [38]:
torch.meshgrid(a,b)

(tensor([[0., 0., 0., 0.],
         [1., 1., 1., 1.],
         [2., 2., 2., 2.],
         [3., 3., 3., 3.],
         [4., 4., 4., 4.]]), tensor([[0.1454, 0.0108, 0.2072, 0.0541],
         [0.1454, 0.0108, 0.2072, 0.0541],
         [0.1454, 0.0108, 0.2072, 0.0541],
         [0.1454, 0.0108, 0.2072, 0.0541],
         [0.1454, 0.0108, 0.2072, 0.0541]]))

## Clamp values in a tensor


In [40]:
a = torch.tensor([4.0, -2.3, 2])
a

tensor([ 4.0000, -2.3000,  2.0000])

In [44]:
a.clamp_(min=0)

tensor([4., 0., 2.])

In [53]:
torch.tensor([1])*4

tensor([4])